## Ways to make it better:



*   One-hot Encoding
*   Better Model





In [0]:
import os

if not os.path.exists("heart-attack-prediction.csv"):
    !rm -rf sample_data
    from google.colab import files
    files.upload()
    !xz -d *.xz
    print("Uploaded files:")
    !ls
else:
    print("Already have files:")
    !ls

Already have files:
heart-attack-prediction.csv


In [0]:
import numpy as np
import pandas as pd

dataset = pd.read_csv("heart-attack-prediction.csv")

# converting all string values to nan
dataset = dataset.convert_objects(convert_numeric=True)

x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 13].values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  import sys


In [0]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = "NaN", strategy ="mean", axis = 0)
imputer = imputer.fit(x[:,0:13])   
x[:, 0:13] = imputer.transform(x[:, 0:13])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
x = sc_X.fit_transform(x)

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x , y , test_size=0.2)

In [0]:
from sklearn.linear_model import LogisticRegression
result = LogisticRegression(random_state = 0)
result.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_predict = result.predict(x_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_predict, y_test)

In [0]:
# Results
print ("Weights:")
print(list(result.coef_*np.std(x, 0)))
print(dataset.columns)
print("\nConfusion Matrix:")
print(cm)

Weights:
[array([ 0.15261953,  0.66499465,  0.65057494, -0.27264159,  0.42547548,
        0.18863831,  0.01771307,  0.07585758,  0.4682554 ,  1.01653247,
        1.01440192,  0.        ,  0.61507881])]
Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'],
      dtype='object')

Confusion Matrix:
[[36  3]
 [ 2 18]]


In [0]:
acc = (cm[0][0]+cm[1][1])/(cm.sum().sum())
prec = (cm[1][1])/(cm[1][0]+cm[1][1])
rec = (cm[1][1])/(cm[0][1]+cm[1][1])
f1 = 2*prec*rec/(prec+rec)
f2 = 5*prec*rec/(4*prec+rec)

metrics = pd.DataFrame({"Metric": ["f1", "f2", "precision", "recall", "accuracy"], "Score": [f1, f2, prec, rec, acc]})
metrics

,Metric,Score
0,f1,0.878049
1,f2,0.865385
2,precision,0.900000
3,recall,0.857143
4,accuracy,0.915254
